In [ ]:
import copy 
import os
import numpy as np
import matplotlib.pyplot as plt
from functools import wraps

from astropy.table import Table, vstack

import pandas as pd
from scipy.optimize import curve_fit

In [ ]:
def get_data(basedir):
    def _read_files(basedir):
        for index, f in enumerate(os.listdir(basedir)):
            if f.endswith('.fits'):
                tt = Table.read(os.path.join(basedir, f), format='fits')
                if len(tt):
                    yield tt
    return vstack(list(_read_files(basedir)))

In [ ]:
simulations = get_data('./simulations')

In [ ]:
good_ring_mask = simulations['good_ring']>0

In [ ]:
simulations

In [ ]:
def plot_spectrum(data, bins=100, range=None, log=True, label=None, fit=False, fit_p0=None, fit_bounds=(-np.inf, np.inf)):
    counts, bin_edges, patches = plt.hist(data['mc_energy'], bins=bins, histtype='step', lw=2, log=log, label=label)
    y_limits = plt.gca().get_ylim()
    if fit:
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        popt, pcov = curve_fit(fit, bin_centers, counts, p0=fit_p0, bounds=fit_bounds)
        plt.plot(bin_centers, fit(bin_centers, *popt), label=f'Fit of {label}\nFit parameters: {popt}')
        plt.ylim(y_limits)
    plt.legend()
    plt.xlim(*range)

plot_spectrum(simulations[good_ring_mask], bins=200, range=(0, 0.1), log=True, label='good_ring', fit=lambda x, a, b: a*x**b)
plot_spectrum(simulations, bins=200, range=(0, 1), log=True, label='all rings', fit=lambda x, a, b: a*x**b)

In [ ]:
def reweight(data, spectral_index):
    weights = data['mc_energy']**spectral_index
    return weights/np.sum(weights)

def plot_reweighted_spectrum(data, spectral_index, bins=100, range=None, log=True, label=None, fit=False, fit_p0=None, fit_bounds=(-np.inf, np.inf)):
    weights = reweight(data, spectral_index)
    counts, bin_edges, patches = plt.hist(data['mc_energy'], bins=bins, histtype='step', lw=2, log=log, label=label, weights=weights)
    y_limits = plt.gca().get_ylim()
    if fit:
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        popt, pcov = curve_fit(fit, bin_centers, counts, p0=fit_p0, bounds=fit_bounds)
        plt.plot(bin_centers, fit(bin_centers, *popt), label=f'Fit of {label}\nFit parameters: {popt}')
        plt.ylim(y_limits)
    plt.legend()
    plt.xlim(*range)

    return counts, bin_edges

reweighted_counts, bin_edges = plot_reweighted_spectrum(simulations[good_ring_mask], -0.7, bins=200, range=(0, 1), log=True, label='Good Rings reweighted to -2.7', fit=lambda x, a, b: a*x**b, fit_p0=(1, -2.7))
counts, bin_edges = plot_reweighted_spectrum(simulations[good_ring_mask], 0, bins=200, range=(0, 1), log=True, label='Good Rings', fit=lambda x, a, b: a*x**b)

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
def reweight_broken(data, energy_bands):
    weights = np.ones(len(data))
    for (e0, e1, spectral_index) in energy_bands:
        mask = (data['mc_energy'] >= e0) & (data['mc_energy'] < e1)
        weights[mask] = data['mc_energy'][mask]**spectral_index
    return weights / np.sum(weights)

def plot_reweighted_quantity(data, quantity, energy_bands=[], bins=100, range=(None, None), log=True, label=None, fit=False, fit_p0=None, fit_bounds=(-np.inf, np.inf)):
    if not energy_bands:
        weights = np.ones(len(data))/len(data)
    else:
        weights = reweight_broken(data, energy_bands)
    
    counts, bin_edges, patches = plt.hist(data[quantity], bins=bins, histtype='step', lw=2, log=log, label=label, weights=weights)
    y_limits = plt.gca().get_ylim()
    
    if fit:
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        popt, pcov = curve_fit(fit, bin_centers, counts, p0=fit_p0, bounds=fit_bounds)
        plt.plot(bin_centers, fit(bin_centers, *popt), label=f'Fit of {label}\nFit parameters: {popt}')
        plt.ylim(y_limits)
    
    plt.legend()
    plt.xlim(*range)
    #plt.show()

# Example usage
# energy_bands = [(0.0, 0.01, -0.12), (0.01, 0.02, -0.46), (0.02, np.inf, -0.7)]  # Define energy bands and corresponding spectral indices
energy_bands = [(0.0, 0.02, -0.46), (0.02, np.inf, -0.7)]  # Define energy bands and corresponding spectral indices

plot_reweighted_quantity(simulations[good_ring_mask], 'ring_radius', bins=200, log=False, label='Good Rings')
plot_reweighted_quantity(simulations[good_ring_mask], 'ring_radius', energy_bands=[(0.0, np.inf, -0.7)], bins=200, log=False, label='Good Rings reweighted to -0.7')
plot_reweighted_quantity(simulations[good_ring_mask], 'ring_radius', energy_bands=energy_bands, bins=200, log=False, label='Good Rings reweighted to different indices')
# plot_reweighted_quantity(simulations[good_ring_mask], 'ring_radius', energy_bands=[(0.0, np.inf, -1.0)], bins=200, log=False, label='Good Rings reweighted to -1.0')
# plot_reweighted_quantity(simulations[good_ring_mask], 'ring_radius', energy_bands=[(0.0, np.inf, -0.12)], bins=200, log=False, label='Good Rings reweighted to -0.12')
# plot_reweighted_quantity(simulations[good_ring_mask], 'ring_radius', energy_bands=[(0.0, np.inf, -0.46)], bins=200, log=False, label='Good Rings reweighted to -0.46')




In [ ]:
plot_reweighted_quantity(simulations[good_ring_mask], 'mc_energy', bins=200, log=True, label='Good Rings')
plot_reweighted_quantity(simulations[good_ring_mask], 'mc_energy', energy_bands=[(0.0, np.inf, -0.7)], bins=200, log=True, label='Good Rings reweighted to -0.7')
plot_reweighted_quantity(simulations[good_ring_mask], 'mc_energy', energy_bands=energy_bands, bins=200, log=True, label='Good Rings reweighted to different indices')

In [ ]:
reweighting_options = [
    [(0.0, np.inf, -0.12),],
    [(0.0, np.inf, -0.46),],
    [(0.0, np.inf, -0.7),],
    [(0.0, 0.02, -0.46), (0.02, np.inf, -0.7)],
]

for reweighting_option in reweighting_options:
    weights = reweight_broken(simulations[good_ring_mask], reweighting_option)
    plt.hist(weights, bins=100, histtype='step', lw=2, log=True, label=f'{reweighting_option}')
    plt.legend()

In [ ]:
for reweighting_option in reweighting_options:
    weights = reweight_broken(simulations[good_ring_mask], reweighting_option)
    plt.scatter(simulations[good_ring_mask]['mc_energy'], weights, label=f'{reweighting_option}')
    plt.legend()

In [ ]:
def reweight_gaug(data, zenith):
    zenith = np.deg2rad(zenith)
    weights = np.ones(len(data))
    weights = data['mc_energy']**-0.7 * (1/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(zenith)/115) + 0.054/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(zenith)/850))
    return weights / np.sum(weights)

def plot_reweighted_gaug(data, zenith, bins=100, range=(None, None), log=True, label=None, fit=False, fit_p0=None, fit_bounds=(-np.inf, np.inf)):
    weights = reweight_gaug(data, zenith)
    counts, bin_edges, patches = plt.hist(data['mc_energy'], bins=bins, histtype='step', lw=2, log=log, label=label, weights=weights)
    y_limits = plt.gca().get_ylim()
    
    if fit:
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        popt, pcov = curve_fit(fit, bin_centers, counts, p0=fit_p0, bounds=fit_bounds)
        plt.plot(bin_centers, fit(bin_centers, *popt), label=f'Fit of {label}\nFit parameters: {popt}')
        plt.ylim(y_limits)
    
    plt.legend()
    plt.xlim(*range)
    #plt.show()

plot_reweighted_gaug(simulations[good_ring_mask], 0, bins=200, log=True, label='Good Rings')
plot_reweighted_gaug(simulations[good_ring_mask], 20, bins=200, log=True, label='Good Rings')

In [ ]:
dat = Table.read('{}'.format(listdir[0]), format='fits')

for muon_file in listdir:
    dat2 = Table.read(muon_file, format='fits')
    
    # Check if 'good_ring' exists and convert to bool
    if 'good_ring' in dat2.colnames:
        dat2['good_ring'] = dat2['good_ring'].astype(bool)
    
    # Stack the tables
    if dat is None:
        dat = dat2
    else:
        dat = vstack([dat, dat2])

df = dat.to_pandas()
df_sim_00043_alignment = df[
    df['good_ring'] & 
    (df['muon_efficiency'] < muon_efficiency_cut) & 
    (df['size_outside'] < size_outside_cut)
]
df_sim_00043_alignment

# Applying cuts

In [ ]:
ring_completeness_cut = 0
ring_containment_cut = 0
min_radius_cut = 0.8
max_radius_cut = 1.5
ring_center_distance_cut = 2
impact_distance_cut = 15


In [ ]:
def calculate_survival_percentage(df, df_cut):
    return (len(df_cut) / len(df)) * 100

def apply_cuts(df, completeness_cut, containment_cut, min_radius_cut, max_radius_cut, center_distance_cut, impact_distance_cut):
    return df[
        (df['ring_completeness'] > completeness_cut) &
        (df['ring_containment'] > containment_cut) &
        (df['ring_radius'] > min_radius_cut) &
        (df['ring_radius'] < max_radius_cut) &
        (df['ring_center_y'].abs() < center_distance_cut) &
        (df['impact_parameter'] < impact_distance_cut)
    ]


# Apply cuts to each dataframe
df_data_cut = apply_cuts(df_good_data_real, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_data_cut: {len(df_data_cut)} ({calculate_survival_percentage(df_good_data_real, df_data_cut):.2f}%) out of {len(df_good_data_real)}")

df_good_data_winter_2024_cut = apply_cuts(df_good_data_winter_2024, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_good_data_winter_2024_cut: {len(df_good_data_winter_2024_cut)} ({calculate_survival_percentage(df_good_data_winter_2024, df_good_data_winter_2024_cut):.2f}%) out of {len(df_good_data_winter_2024)}")

df_sim_00043_cut = apply_cuts(df_sim_00043_alignment, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_00043_cut: {len(df_sim_00043_cut)} ({calculate_survival_percentage(df_sim_00043_alignment, df_sim_00043_cut):.2f}%) out of {len(df_sim_00043_alignment)}")

df_sim_0015_cut = apply_cuts(df_sim_0015_alignment, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0015_cut: {len(df_sim_0015_cut)} ({calculate_survival_percentage(df_sim_0015_alignment, df_sim_0015_cut):.2f}%) out of {len(df_sim_0015_alignment)}")

df_sim_0015_cut_3200m = apply_cuts(df_sim_0015_alignment_3200m, ring_completeness_cut, ring_containment_cut, min_radius_cut, max_radius_cut, ring_center_distance_cut, impact_distance_cut)
print(f"Events survived for df_sim_0015_alignment_3200m_cut: {len(df_sim_0015_cut_3200m)} ({calculate_survival_percentage(df_sim_0015_alignment_3200m, df_sim_0015_cut_3200m):.2f}%) out of {len(df_sim_0015_alignment_3200m)}")

# Functions for weighting

In [ ]:
def weighted_data(df, power_index):
    df_copy = copy.deepcopy(df[['ring_radius', 'mc_energy', 'ring_completeness', 'ring_containment', 'impact_parameter']])
    df_copy['weights'] = df_copy['mc_energy'] ** (-power_index)
    return df_copy

In [ ]:
def calculate_weights_broken_power_law(df, energy_breaks, power_indices):
    """
    Calculate weights using a broken power law.
    
    Parameters:
    - df (pd.DataFrame): DataFrame with at least 'mc_energy' column.
    - energy_breaks (list): List of energy breakpoints for the broken power law.
    - power_indices (list): List of power indices for each energy segment.
    
    Returns:
    - pd.DataFrame: A DataFrame with weights added.
    """
    if len(energy_breaks) + 1 != len(power_indices):
        raise ValueError("The number of power indices must be one more than the number of energy breaks.")
    
    df_copy = df.copy()
    df_copy['weights'] = np.nan

    # Assign weights based on the energy ranges
    for i, power_index in enumerate(power_indices):
        if i == 0:
            mask = df_copy['mc_energy'] < energy_breaks[0]
            print(f"LEN MASK {sum(mask)}")
        elif i == len(power_indices) - 1:
            mask = df_copy['mc_energy'] >= energy_breaks[-1]
            print(f"LEN MASK {sum(mask)}")
        else:
            mask = (df_copy['mc_energy'] >= energy_breaks[i - 1]) & (df_copy['mc_energy'] < energy_breaks[i])
            print(f"LEN MASK {sum(mask)}")
        
        df_copy.loc[mask, 'weights'] = df_copy.loc[mask, 'mc_energy'] ** (-power_index)

    return df_copy


# Applying weights

In [ ]:
energy_breaks = [0.01, 0.02]
power_indices = [0.12, 0.46, 0.71]

df_sim_broken_power_law_weight_all_3200m = calculate_weights_broken_power_law(
    df_sim_0015_cut_3200m, energy_breaks, power_indices
)

In [ ]:
df_sim_broken_power_law_weight_all_3200m['weights']

In [ ]:
print(7.897**(-1/0.71))
print(1.74 ** (-1/0.12))

In [ ]:
df_sim_broken_power_law_weight_all_3200m['mc_energy']

In [ ]:
df_sim_standard_power_law_weigth = weighted_data(df_sim_0015_cut, 0.7)

df_test = weighted_data(df_sim_0015_cut_3200m, 0.7)

df_sim_broken_power_law_weight_016 = weighted_data(df_sim_0015_cut[df_sim_0015_cut['mc_energy'] < 0.01], 0.12)
df_sim_broken_power_law_weight_046 = weighted_data(df_sim_0015_cut[(df_sim_0015_cut['mc_energy'] > 0.01) & (df_sim_0015_cut['mc_energy'] < 0.02)], 0.46)
#df_sim_broken_power_law_weight_046 = weighted_data(df_sim_0015_cut[(df_sim_0015_cut['mc_energy'] < 0.02)], 0.46)
df_sim_broken_power_law_weight_071 = weighted_data(df_sim_0015_cut[df_sim_0015_cut['mc_energy'] >= 0.02], 0.71)

df_sim_broken_power_law_weight_all = pd.concat([df_sim_broken_power_law_weight_046, df_sim_broken_power_law_weight_071], ignore_index=True)

In [ ]:
ring_completeness_cut_hist = 0
bin_number = 100
density_param = True

min_value = min_radius_cut
max_value = max_radius_cut

# Define the bin edges
bins = np.linspace(min_value, max_value, bin_number)  

plt.figure(figsize=(11,8))

# print(f"Size of df_sim_standard_power_law_weigth: {len(df_sim_standard_power_law_weigth)}")
# print(f"Size of df_sim_broken_power_law_weight_all: {len(df_sim_broken_power_law_weight_all)}")
print(f"Size of df_sim_0015_cut: {len(df_sim_0015_cut)}")

# z,x,c = plt.hist(df_sim_standard_power_law_weigth['ring_radius'][
#                  df_sim_standard_power_law_weigth['ring_completeness'] > ring_completeness_cut_hist], 
#                  weights=df_sim_standard_power_law_weigth['weights'][df_sim_standard_power_law_weigth['ring_completeness'] > ring_completeness_cut_hist],
#                  bins=bins, 
#                  label='Standard Reweighted Simulation', 
#                  histtype='step', 
#                  density=density_param, 
#                  lw=3
#                 )

# z,x,c = plt.hist(df_sim_broken_power_law_weight_all['ring_radius'], weights=df_sim_broken_power_law_weight_all['weights'], bins=bin_number, label='Broken Power Law Reweighted Simulation',
#          alpha=0.5, histtype='step', density=density_param, lw=3)

# z,x,c = plt.hist(df_sim_0015_cut['ring_radius'][df_sim_0015_cut['ring_completeness'] > ring_completeness_cut_hist], bins=bins, label='Original Simulation with cut',
#          alpha=0.5, density=density_param)

# z,x,c = plt.hist(df_sim_0015_cut['ring_radius'], bins=bins, label='Original Simulation',
#          alpha=0.7, density=density_param, histtype='step', lw=3)

z,x,c = plt.hist(df_data_cut['ring_radius'][df_data_cut['ring_completeness'] > ring_completeness_cut_hist], bins=bins, label='All 2024 year data',
         alpha=0.7, density=density_param, histtype='step', lw=3)

# z,x,c = plt.hist(df_good_data_winter_2024_cut['ring_radius'][df_good_data_winter_2024_cut['ring_completeness'] > ring_completeness_cut_hist], bins=bins, label='Winter 2024 + zenith angle < 20 deg + good atmo conditions',
#          alpha=0.5, density=density_param)

z,x,c = plt.hist(df_sim_0015_cut_3200m['ring_radius'], bins=bins, label='Simulations for start height 3200m',
         alpha=0.5, density=density_param)

z,x,c = plt.hist(df_test['ring_radius'], weights=df_test['weights'], bins=bins, label='Simulations for start height 3200m PL reweighted',
         alpha=0.5, density=density_param, histtype='step', lw=3)

z,x,c = plt.hist(df_test['ring_radius'], weights=df_sim_broken_power_law_weight_all_3200m['weights'], bins=bins, label='Simulations for start height 3200m Broken PL reweighted',
         alpha=0.5, density=density_param, histtype='step', lw=3)

plt.xlabel('Ring Radius [deg]')
plt.axvline(1.2157382060313375, lw = 2, ls = '-.', c = 'r', label = 'summer max ring radius on 2 km',alpha=0.75)
plt.axvline(1.2354433340796351, lw = 2, ls = '-.', c = 'k', label = 'winter max ring radius on 2 km',alpha=0.75)
plt.ylabel('Normalized Count')
#plt.title('Histogram of Radius with Reweighted Energy Distribution')
plt.xlim(0.9,1.3)
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,7))
he_muons = df_sim_0015_cut[df_sim_0015_cut['mc_energy'] > 0.02]
GeV_10_muons = df_sim_0015_cut[df_sim_0015_cut['mc_energy'] < 0.01]
GeV_10_20_muons = df_sim_0015_cut[(df_sim_0015_cut['mc_energy'] > 0.01) & (df_sim_0015_cut['mc_energy'] < 0.02)]
z,x,c = plt.hist(he_muons['ring_radius'], bins = 100, label='High Energy Muons')
z,x,c = plt.hist(GeV_10_muons['ring_radius'], bins = 100, label='Muons < 10 GeV')
z,x,c = plt.hist(GeV_10_20_muons['ring_radius'], bins = 100, label='10 GeV < Muons < 20 GeV')
plt.xlabel('Radius')
plt.ylabel('Normalized Count')
plt.title('Histogram of Radius with Reweighted Energy Distribution')
plt.xlim(0.9,1.3)
#plt.ylim(0,1)
plt.legend()
plt.show()

In [ ]:
bin_number_hist = 50
IP_cut = 3
plt.figure(figsize=(10,7))
z,x,c = plt.hist(df_good_data_winter_2024_cut['ring_radius'], bins = bin_number_hist, label='Winter 2024 + zenith angle < 20 deg + good atmo conditions', density=True)
z,x,c = plt.hist(df_good_data_winter_2024_cut['ring_radius'][df_good_data_winter_2024_cut['impact_parameter'] < 9.8], bins = bin_number_hist, label='+ IP<9.8m', density=True, histtype='step', lw=3)
z,x,c = plt.hist(df_good_data_winter_2024_cut['ring_radius'][
    (df_good_data_winter_2024_cut['impact_parameter'] < IP_cut) & 
    (df_good_data_winter_2024_cut['ring_center_x'] < 0.5) & 
    (df_good_data_winter_2024_cut['ring_center_y'] < 0.5)], 
    bins = bin_number_hist, 
    label=f'+ IP<{IP_cut}m + RC < 0.5deg', 
    density=True, 
    histtype='step', 
    lw=3
    )
plt.legend()
plt.xlim(0.6, 1.5)
plt.xlim(0.6, 1.5)

In [ ]:
z,x,c = plt.hist(df_sim_0015_cut['impact_parameter'], bins = 100, label='Simulation')
plt.axvline(5.5, c = 'r')
plt.axvline(5.6, c = 'r')

In [ ]:
len(df_sim_0015_cut['ring_radius'])

In [ ]:
len(df_sim_0015_with_IP_cut['ring_radius']) - len(df_sim_0015_cut['ring_radius'])

In [ ]:
plt.figure(figsize=(10,7))
df_sim_0015_with_IP_cut = df_sim_0015_cut[(df_sim_0015_cut['impact_parameter'] < 5.5) | (df_sim_0015_cut['impact_parameter'] > 5.6)]

z,x,c = plt.hist(df_sim_0015_with_IP_cut['ring_radius'], bins = 100, label='Simulation with IP cut')
#z,x,c = plt.hist(df_good_data_winter_2024_cut['ring_radius'], bins = 100, label='Simulation with IP cut')
z,x,c = plt.hist(df_sim_0015_cut['ring_radius'], bins = 100, label='Simulation without IP cut', histtype='step', lw=3)
plt.legend()


In [ ]:
z,x,c = plt.hist(df_sim_0015_with_IP_cut['impact_parameter'], bins = 100, label='Simulation with IP cut')

In [ ]:
plt.hist(he_muons[he_muons['ring_radius'] < 1.17]['ring_completeness'], bins = 25)

In [ ]:
plt.figure(figsize=(10,8))

# Calculate the weights
df_sim_0015_cut['weights'] = df_sim_0015_cut['mc_energy'] ** -0.7

# Determine the range for bins based on all datasets
min_value = min(df_sim_0015_cut['ring_radius'].min(), df_data_cut['ring_radius'].min())
max_value = max(df_sim_0015_cut['ring_radius'].max(), df_data_cut['ring_radius'].max())

# Define the bin edges
bins = np.linspace(min_value, max_value, 31)  # 30 bins between min_value and max_value

# Plot the histogram of radius with the calculated weights
plt.hist(df_sim_0015_cut['ring_radius'], weights=df_sim_0015_cut['weights'],
         bins=bins, label='Reweighted Simulation', histtype='step', density=True, lw=2)

plt.hist(df_sim_0015_cut['ring_radius'], bins=bins, label='Original Simulation',
         alpha=0.5, density=True)

plt.hist(df_data_cut['ring_radius'], bins=bins, label='Data', alpha=0.5, density=True)

plt.xlabel('Radius')
plt.ylabel('Normalized Count')
plt.title('Histogram of Radius with Reweighted Energy Distribution')
plt.legend()
plt.show()

In [ ]:
# plt.figure(figsize=(10,8))

# # Calculate the weights
# df_sim_0015_cut['weights'] = df_sim_0015_cut['mc_energy'] ** -0.7

# # Determine the range for bins based on all datasets
# min_value = min(df_sim_0015_cut['ring_radius'].min(), df_data_cut['ring_radius'].min())
# max_value = max(df_sim_0015_cut['ring_radius'].max(), df_data_cut['ring_radius'].max())

# # Define the bin edges
# bins = np.linspace(min_value, max_value, 31)  # 30 bins between min_value and max_value

# # Plot the histogram of radius with the calculated weights
# plt.hist(df_sim_0015_cut['ring_radius'], weights=df_sim_0015_cut['weights'],
#          bins=bins, label='Reweighted Simulation', histtype='step', density=True, lw=3)

# plt.hist(df_data_cut['ring_radius'], bins=bins, label='Data', alpha=0.5, density=True, color = 'green')

# plt.xlabel('Ring Radius')
# plt.ylabel('Normalized Count')
# plt.title('Histogram of Radius with Reweighted Energy Distribution for physicaly justified ring radius range')
# plt.legend()
# plt.show()

In [ ]:
plt.hist(df_sim_0015_cut['mc_energy'], log=True)

In [ ]:
min_value = min(df_sim_0015_cut['ring_radius'].min(), df_data_cut['ring_radius'].min())
max_value = max(df_sim_0015_cut['ring_radius'].max(), df_data_cut['ring_radius'].max())


bins = np.linspace(min_value, max_value, 31) 

plt.figure(figsize=(10,8))
df_sim_0015_cut['weights'] = df_sim_0015_cut['mc_energy'] ** -0.46
plt.hist(df_sim_0015_cut['ring_radius'][df_sim_0015_cut['mc_energy'] < 0.02], weights=df_sim_0015_cut['weights'][df_sim_0015_cut['mc_energy'] < 0.02], bins=bins, label='reweighting', histtype='step', density=True, lw = 2)
plt.hist(df_sim_0015_cut['ring_radius'], bins=bins, label='no reweighting', alpha = 0.5, density=True)
plt.hist(df_data_cut['ring_radius'], bins=bins, label='data', alpha = 0.5, density=True)
plt.xlabel('Radius')
plt.ylabel('Weighted Count')
plt.title('Histogram of Radius with Reweighted Energy Distribution')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,8))

GeV_10_muons = df_sim_0015_cut[df_sim_0015_cut['mc_energy'] < 0.01][:34000000]
GeV_20_muons = df_sim_0015_cut[(df_sim_0015_cut['mc_energy'] > 0.01) & (df_sim_0015_cut['mc_energy'] < 0.02)][:21750000]
GeV_all_muons = df_sim_0015_cut[df_sim_0015_cut['mc_energy'] > 0.02][:952000000]

min_value = min(df_sim_0015_cut['ring_radius'].min(), df_data_cut['ring_radius'].min())
max_value = max(df_sim_0015_cut['ring_radius'].max(), df_data_cut['ring_radius'].max())

bins = np.linspace(min_value, max_value, 31) 
bins = 30

GeV_10_muons['weights'] = GeV_10_muons['mc_energy'] ** -0.16
GeV_20_muons['weights'] = GeV_20_muons['mc_energy'] ** -0.46
GeV_all_muons['weights'] = GeV_all_muons['mc_energy'] ** -0.16
plt.hist(GeV_10_muons['ring_radius'], bins=bins, label='< 10 GeV', weights=GeV_10_muons['weights'], alpha = 0.5, density=True)
plt.hist(GeV_20_muons['ring_radius'], bins=bins, label='10-20 GeV', weights=GeV_20_muons['weights'], alpha = 0.5, density=True)
plt.hist(GeV_all_muons['ring_radius'], bins=bins, label='20+ GeV', weights=GeV_all_muons['weights'], alpha = 0.5, density=True)
#plt.hist(df_sim_0015_cut['ring_radius'], bins=bins, label='no reweighting', alpha = 0.5, density=True)
plt.hist(df_data_cut['ring_radius'], bins=bins, label='data', alpha = 0.5, density=True)
plt.xlabel('Radius')
plt.ylabel('Weighted Count')
plt.title('Histogram of Radius with Reweighted Energy Distribution')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
new_weights = pd.concat([
    GeV_10_muons['weights'],
    GeV_20_muons['weights'],
    GeV_all_muons['weights']
], ignore_index=True)
all_muons = pd.concat([
    GeV_10_muons,
    GeV_20_muons,
    GeV_all_muons
], ignore_index=True)

plt.hist(all_muons['ring_radius'], bins=bins, label='Broken Power Law weighting', weights=new_weights, alpha = 0.9, density=True, histtype='step', lw = 2)
#plt.hist(df_sim_0015_cut['ring_radius'], bins=bins, label='no reweighting', alpha = 0.5, density=True)
plt.hist(df_data_cut['ring_radius'], bins=bins, label='data', alpha = 0.5, density=True)
plt.xlabel('Radius')
plt.ylabel('Weighted Count')
plt.title('Histogram of Radius with Reweighted Energy Distribution')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,8))

def get_power_index(mc_energy):
    if mc_energy < 0.01:
        return -0.16
    elif 0.01 <= mc_energy < 0.02:
        return -0.46
    else:
        return -0.7

# Apply the power index to calculate weights
df_sim_0015_cut['weights'] = df_sim_0015_cut['mc_energy'].apply(lambda x: x ** get_power_index(x))
plt.hist(df_sim_0015_cut['ring_radius'], weights=df_sim_0015_cut['weights'], bins=30, label='reweighting', histtype='step', density=True, lw = 2)
plt.hist(df_sim_0015_cut['ring_radius'], bins=30, label='no reweighting', alpha = 0.5, density=True)
plt.hist(df_data_cut['ring_radius'], bins=30, label='data', alpha = 0.5, density=True)
plt.xlabel('Radius')
plt.ylabel('Weighted Count')
plt.title('Histogram of Radius with Reweighted Energy Distribution')
plt.legend()
plt.show()

In [ ]:
df_sim_0015_cut

In [ ]:
mc_energy = df_sim_0015_cut['mc_energy']

conditions = [
    mc_energy < 0.01,
    (mc_energy >= 0.01) & (mc_energy < 0.02),
    mc_energy >= 0.02
]
choices = [-0.16, -0.46, -0.7]
exponents = np.select(conditions, choices, default=-0.7)

df_sim_0015_cut['weights'] = mc_energy ** exponents

plt.figure(figsize=(10,8))

plt.hist(df_sim_0015_cut['ring_radius'], weights=df_sim_0015_cut['weights'], bins=30, label='reweighting', histtype='step', density=True, lw = 2)
plt.hist(df_sim_0015_cut['ring_radius'], weights=df_sim_0015_cut['weights'], bins=30, label='broken power law', histtype='step', density=True, lw = 2)
plt.hist(df_sim_0015_cut['ring_radius'], bins=30, label='no reweighting', alpha = 0.5, density=True)
#plt.hist(df_data_cut['ring_radius'], bins=30, label='data', alpha = 0.5, density=True)

plt.xlabel('Radius')
plt.ylabel('Weighted Count')
plt.title('Histogram of Radius with Reweighted Energy Distribution')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
mc_energy = df_sim_0015_cut['mc_energy']

conditions = [
    mc_energy < 0.01,
    (mc_energy >= 0.01) & (mc_energy < 0.02),
    mc_energy >= 0.02
]

choices = [-0.16, -0.46, -0.7]
exponents = np.select(conditions, choices, default=-0.7)

df_sim_0015_cut['broken_weights'] = mc_energy ** exponents

df_sim_0015_cut['weights'] = df_sim_0015_cut['mc_energy'] ** -0.7

z,x,c = plt.hist(df_sim_0015_cut['mc_energy'], log = True, alpha = 0.5, bins = 2000)
z,x,c = plt.hist(df_sim_0015_cut['mc_energy'], weights=df_sim_0015_cut['weights'], log=True,  histtype='step', label='weighted to -2.7', lw = 3, bins = 2000)
z,x,c = plt.hist(df_sim_0015_cut['mc_energy'], weights=df_sim_0015_cut['broken_weights'], log=True,  histtype='step', label='broken power law', lw = 3, bins = 2000)
plt.xlim(0,0.03)
plt.legend()

In [ ]:
mc_energy = df_sim_0015_cut['mc_energy']
conditions = [
    mc_energy < 0.01,
    (mc_energy >= 0.01) & (mc_energy < 0.02),
    mc_energy >= 0.02
]
choices = [-0.16, -0.46, -0.7]
exponents = np.select(conditions, choices, default=-0.7)
df_sim_0015_cut['weights'] = mc_energy ** exponents
df_sim_0015_cut['weights']

In [ ]:
df_sim_0015_cut['weights'] = df_sim_0015_cut['mc_energy'] ** -0.7
df_sim_0015_cut['weights']

# Ring Radius vs Completeness

In [ ]:
plt.scatter(df_data_cut['ring_completeness'],df_data_cut['ring_radius'], s = 1)

In [ ]:
plt.scatter(df_sim_0015_cut['ring_completeness'],df_sim_0015_cut['ring_radius'], s = 1)

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist2d(df_data_cut['ring_radius'], df_data_cut['ring_completeness'], bins=100, cmap='viridis')
plt.colorbar(label='Count')
plt.xlabel('Ring Radius')
plt.ylabel('Ring Completeness')
plt.title('2D Histogram of Ring Radius vs. Ring Completeness')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist2d(df_sim_0015_cut['ring_radius'], df_sim_0015_cut['ring_completeness'], bins=100, cmap='viridis')
plt.colorbar(label='Count')
plt.xlabel('Ring Radius')
plt.ylabel('Ring Completeness')
plt.title('2D Histogram of Ring Radius vs. Ring Completeness')
plt.show()

In [ ]:
z,x,c = plt.hist(df_data_cut['ring_completeness'], bins = 100, density=True)
z,x,c = plt.hist(df_sim_0015_cut['ring_completeness'], bins = 100, density=True, alpha = 0.5)

In [ ]:
z,x,c = plt.hist(df_data_cut['ring_completeness'][df_data_cut['ring_completeness'] > 0.6], bins = 100, density=True)
z,x,c = plt.hist(df_sim_0015_cut['ring_completeness'][df_sim_0015_cut['ring_completeness'] > 0.6], bins = 100, density=True, alpha = 0.5)